In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import numpy as np
import time
import os
from datetime import datetime

from model_final_2 import Model
from configs_final_2 import config
from tensorflow.core.protobuf import rewriter_config_pb2
from sklearn.model_selection import train_test_split

In [ ]:
def data_loader_gan():
    time_bin = 700
    X_test = np.load("X_test.npy")
    X_test = X_test[:,0:22,0:time_bin]
    y_test = np.load("y_test.npy")
    person_test = np.load("person_test.npy")

    X_train_valid = np.load("X_train_valid.npy")
    X_train_valid = X_train_valid[:,0:22,0:time_bin]
    y_train_valid = np.load("y_train_valid.npy")
    person_train_valid = np.load("person_train_valid.npy")
    
    X_train_gan = np.load("X_train_gan4.npy")
    X_train_gan = X_train_gan[:,0:22,0:time_bin,:]
    shp = X_train_gan.shape
    X_train_gan = np.reshape(X_train_gan,(shp[0],shp[1],shp[2]))
    y_train_gan = np.load("y_train_gan4.npy")
    
    N_train = X_train_valid.shape[0]
    idx_train_valid = np.arange(N_train,dtype='int')

    _, X_val, _ , idx_val = train_test_split(X_train_valid, idx_train_valid, test_size=0.2, random_state=21)

    y_val = y_train_valid[idx_val]
    person_val = person_train_valid[idx_val]
    
    idx11 = np.arange(0,X_test.shape[2],2,dtype=int)
    idx21 = np.arange(1,X_test.shape[2],2,dtype=int)
    X_test_ss1 = np.take(X_test,idx11,axis=2)
    X_test_ss2 = np.take(X_test,idx21,axis=2)
    X_test = np.concatenate((X_test_ss1,X_test_ss2),axis=0)
    y_test = np.concatenate((y_test,y_test),axis=0)
    person_test = np.concatenate((person_test,person_test),axis=0)
    
    idx12 = np.arange(0,X_train_gan.shape[2],2,dtype=int)
    idx22 = np.arange(1,X_train_gan.shape[2],2,dtype=int)
    X_train_ss1 = np.take(X_train_gan,idx12,axis=2)
    X_train_ss2 = np.take(X_train_gan,idx22,axis=2)
    X_train_gan = np.concatenate((X_train_ss1,X_train_ss2),axis=0)
    y_train_gan = np.concatenate((y_train_gan,y_train_gan),axis=0)
    
    idx13 = np.arange(0,X_val.shape[2],2,dtype=int)
    idx23 = np.arange(1,X_val.shape[2],2,dtype=int)
    X_val_ss1 = np.take(X_val,idx13,axis=2)
    X_val_ss2 = np.take(X_val,idx23,axis=2)
    X_val = np.concatenate((X_val_ss1,X_val_ss2),axis=0)
    y_val = np.concatenate((y_val,y_val),axis=0)
    person_val = np.concatenate((person_val,person_val),axis=0)

    tl = X_train_gan.shape[2]
    X_test = np.reshape(X_test,(-1,22,tl,1))
    X_train_gan = np.reshape(X_train_gan,(-1,22,tl,1))
    X_val = np.reshape(X_val,(-1,22,tl,1))

    label0 = 769
    new_label0 = 0
    for i in range(4):
        m1 = (y_test==label0)
        m3 = (y_val==label0)
        np.place(y_test,m1,new_label0)
        np.place(y_val,m3,new_label0)
        label0 += 1
        new_label0 += 1

    labelNames = [0,1,2,3]
    train_set = {'data': X_train_gan, 'labels': y_train_gan}
    test_set = {'data': X_test, 'labels': y_test}
    val_set = {'data': X_val, 'labels': y_val}
    return train_set, test_set, val_set, labelNames

def sample_batch(dataset, batch_size):
    N = dataset['data'].shape[0]
    indices = np.random.randint(N, size=batch_size)
    return {key: dataset[key][indices] for key in dataset}

def train_model_gan(target_dir,start,stop,step,time_bin,fsz,use_batchnorm,use_dropout,use_l2loss,num_class,max_step,log_frequency,batch_size,model_saving_freq):
    # load data
    print("Loading datasets...")
    trainSet, testSet, valSet, labelNames = data_loader_gan()
    print("Dataset loading completes.")
    
    best_val_acc = 0
    best_val_path = target_dir
    


    # reset default graph
    tf.reset_default_graph()

    global_step = tf.train.get_or_create_global_step()
    
    model = Model(config, labelNames,start,stop,step,time_bin,fsz,use_batchnorm,use_dropout,use_l2loss)

    # training setups
    saver = tf.train.Saver(max_to_keep=100)

    # Build an initialization operation to run below.
    init = tf.global_variables_initializer()
    
    config_proto = tf.ConfigProto()

    off = rewriter_config_pb2.RewriterConfig.OFF
    config_proto.graph_options.rewrite_options.arithmetic_optimization = off
    
    sess = tf.Session(config=config_proto)

    sess.run(init)

    # main loop
    for step in range(max_step):
        start_time = time.time()

        # prepare data
        train_batch = sample_batch(trainSet, batch_size)
        # feed dict
        feed_dict = {
            model.input: train_batch['data'],
            model.fine_labels: train_batch['labels'],
            model.is_training: True,
            model.drprob: 0.7
        }
        
        fetch_list = [model.optimizer_op, model.loss, model.global_step]
        _, loss_value, global_step_value = sess.run(fetch_list, feed_dict=feed_dict)
        duration = time.time() - start_time
        assert not np.isnan(loss_value), 'Model diverged with loss = NaN'

        # log
        if step % log_frequency == 0:
            num_examples_per_step = batch_size
            examples_per_sec = num_examples_per_step / duration
            sec_per_batch = duration
            format_str = (
                '%s: step %d, examples %d, loss = %.9f (%.3f examples/sec; %.3f sec/batch)'
            )
            print(
                format_str % (
                    datetime.now(), step, batch_size * step,
                    loss_value,
                    examples_per_sec, sec_per_batch
                )
            )

        # Save the model checkpoint periodically.
        if step % model_saving_freq == 0 or (step + 1) == max_step:
            fetch_list = [model.accuracy,model.top_2_accuracy]
            feed_dict = {
                model.input: valSet['data'],
                model.fine_labels: valSet['labels'],
                model.is_training: False,
            }
            val_acc_1, val_acc_2 =  sess.run(fetch_list, feed_dict=feed_dict)
            checkpoint_path = os.path.join(target_dir, 'model.ckpt')
            saver.save(sess, checkpoint_path, global_step=int(global_step_value))
            print('Top 1 validation accuracy: {} and top 2 validation accuracy: {}'.format(val_acc_1,val_acc_2))
            print('Model Saved!')
            if val_acc_1>best_val_acc:
                best_val_acc = val_acc_1
                best_val_path = checkpoint_path+'-'+str(int(global_step_value))
    return best_val_acc,best_val_path


def test_model_gan(model_dir,start,stop,step,time_bin,fsz,use_batchnorm,use_dropout,use_l2loss):
    print("Loading datasets...")
    trainSet, testSet, valSet, labelNames = data_loader_gan()
    print("Dataset loading completes.")
    
    # reset default graph
    tf.reset_default_graph()
    
    # create model
    
    model = Model(config, labelNames,start,stop,step,time_bin,fsz,use_batchnorm,use_dropout,use_l2loss)
    # training setups
    saver = tf.train.Saver(max_to_keep=100)
    test_accuracy = [0, 0]
    
    config_proto = tf.ConfigProto()

    off = rewriter_config_pb2.RewriterConfig.OFF
    config_proto.graph_options.rewrite_options.arithmetic_optimization = off

    with tf.Session(config=config_proto) as session:
        session.run(tf.global_variables_initializer())
        saver.restore(session, model_dir)
        # test model
        exp_results = run_single_step(session, model, testSet, mode='test')
        test_accuracy[0] = exp_results['top_1_accuracy']
        test_accuracy[1] = exp_results['top_2_accuracy']
        print('top_1_accuracy_test = ', test_accuracy[0], 'top_2_accuracy_test = ', test_accuracy[1])
        
    top1_acc = test_accuracy[0]
    top2_acc = test_accuracy[1]
    return top1_acc,top2_acc


def run_single_step(
        session,
        model,
        batch,
        mode='test',
        log=True,
):
    # construct feed dict
    feed_dict = {
        model.input: batch['data'],
        # model.coarse_labels: batch['coarse_labels'],
        model.fine_labels: batch['labels'],
        # model.label_mapping: label_mapping,
        model.is_training: mode == 'train'
    }
    
    # select proper summary op
    if mode == 'train':
        summary_op = model.train_summary_op
    elif mode == 'val':
        summary_op = model.val_summary_op
    else:
        summary_op = model.test_summary_op
    
    # construct fetch list
    fetch_list = [model.global_step, summary_op, model.loss, model.accuracy, model.top_2_accuracy]

    # run single step
    _start_time = time.time()
    _step, _summary, _loss, _top_1, _top_2 = session.run(fetch_list, feed_dict=feed_dict)[:5]
    _end_time = time.time()
    
    # collect step statistics
    step_time = _end_time - _start_time
    batch_size = batch['data'].shape[0]
    
    # log in console
    if log:
        print(('[{:5s} step {:4d}] loss: {:.5f}; top_1_accuracy: {:.5f}; top_5_accuracy: {:5f} ' +
              '({:.3f} sec/batch; {:.3f} instances/sec)'
              ).format(mode, _step, _loss, _top_1, _top_2, 
                       step_time, batch_size / step_time))
    
    # log results to file and return statistics
    if mode == 'test':
        test_fetch_list = [model.per_class_accuracy,
                model.top_2_per_class_accuracy,
                model.confusion_matrix, 
                model.pred, model.probs]
        _top_1_c,  _top_2_c, _cm, _pred, _probs = \
                session.run(test_fetch_list, feed_dict=feed_dict)
        
        
        # Log detailed test results in pickle format
        stats = {
            "loss": _loss,
            "top_1_accuracy": _top_1,
            "top_2_accuracy": _top_2,
            "top_1_perclass_accuracy": _top_1_c,
            "top_2_perclass_accuracy": _top_2_c,
            "confusion_matrix": _cm,
            "pred": _pred,
            "probs": _probs
        }
    else:
        stats = {
            "step": _step,
            "loss": _loss,
            "top_1_accuracy": _top_1,
            "top_2_accuracy": _top_2
        }
        
    return stats


In [ ]:
best_val_gan = {}
num_cls = 4
mstp = 500
lfrq = 10
bsz = 50
msf = 50
tr = './trained_model_final/DCNN_GAN4'
if not os.path.exists(tr):
    os.mkdir(tr)
start = 0
stop = 250
step = 10
time_bin = 350
fsz = 3
use_batchnorm = True
use_dropout = True
use_l2loss = True
acc = 'Accuracy_gan'
path = 'Path_gan'

best_val_gan[acc],best_val_gan[path] = train_model_gan(tr,start,stop,step,time_bin,fsz,use_batchnorm,use_dropout,use_l2loss,num_cls,mstp,lfrq,bsz,msf)

In [ ]:
# Test model accuracy
start = 0
stop = 250
step = 10
time_bin = 350
#Nb = BATCH_SIZE
fsz = 3
use_batchnorm = True
use_dropout = True
use_l2loss = True
path = 'Path_gan' 
model_dir = best_val_gan[path]
print('Test Accuracy for all subjects:')
top_1_acc_test, top_2_acc_test = test_model_gan(model_dir,start,stop,step,time_bin,fsz,use_batchnorm,use_dropout,use_l2loss)

In [2]:
def data_loader_gan2():
    time_bin = 700
    X_test = np.load("X_test.npy")
    X_test = X_test[:,0:22,0:time_bin]
    y_test = np.load("y_test.npy")
    person_test = np.load("person_test.npy")

    X_train_valid = np.load("X_train_valid.npy")
    X_train_valid = X_train_valid[:,0:22,0:time_bin]
    y_train_valid = np.load("y_train_valid.npy")
    person_train_valid = np.load("person_train_valid.npy")
    
    X_train_gan = np.load("X_train_gan4.npy")
    X_train_gan = X_train_gan[:,0:22,0:time_bin,:]
    shp = X_train_gan.shape
    X_train_gan = np.reshape(X_train_gan,(shp[0],shp[1],shp[2]))
    y_train_gan = np.load("y_train_gan4.npy")
    
    N_train = X_train_valid.shape[0]
    idx_train_valid = np.arange(N_train,dtype='int')
    
    N_train_gan = X_train_gan.shape[0]
    idx_train_gan = np.arange(N_train_gan,dtype='int')

    X_train_2, _, idx_train_2 , _ = train_test_split(X_train_gan, idx_train_gan, test_size=0.7, random_state=15)
    X_train_1, X_val, idx_train_1 , idx_val = train_test_split(X_train_valid, idx_train_valid, test_size=0.1, random_state=21)

    y_val = y_train_valid[idx_val]
    person_val = person_train_valid[idx_val]
    
    y_train_1 = y_train_valid[idx_train_1]
    y_train_2 = y_train_gan[idx_train_2]
    
    X_train = np.concatenate((X_train_1,X_train_2),axis=0)
    y_train = np.concatenate((y_train_1,y_train_2),axis=0)
    
    idx11 = np.arange(0,X_test.shape[2],2,dtype=int)
    idx21 = np.arange(1,X_test.shape[2],2,dtype=int)
    X_test_ss1 = np.take(X_test,idx11,axis=2)
    X_test_ss2 = np.take(X_test,idx21,axis=2)
    X_test = np.concatenate((X_test_ss1,X_test_ss2),axis=0)
    y_test = np.concatenate((y_test,y_test),axis=0)
    person_test = np.concatenate((person_test,person_test),axis=0)
    
    idx12 = np.arange(0,X_train.shape[2],2,dtype=int)
    idx22 = np.arange(1,X_train.shape[2],2,dtype=int)
    X_train_ss1 = np.take(X_train,idx12,axis=2)
    X_train_ss2 = np.take(X_train,idx22,axis=2)
    X_train = np.concatenate((X_train_ss1,X_train_ss2),axis=0)
    y_train = np.concatenate((y_train,y_train),axis=0)
    
    idx13 = np.arange(0,X_val.shape[2],2,dtype=int)
    idx23 = np.arange(1,X_val.shape[2],2,dtype=int)
    X_val_ss1 = np.take(X_val,idx13,axis=2)
    X_val_ss2 = np.take(X_val,idx23,axis=2)
    X_val = np.concatenate((X_val_ss1,X_val_ss2),axis=0)
    y_val = np.concatenate((y_val,y_val),axis=0)
    person_val = np.concatenate((person_val,person_val),axis=0)

    tl = X_train.shape[2]
    X_test = np.reshape(X_test,(-1,22,tl,1))
    X_train = np.reshape(X_train,(-1,22,tl,1))
    X_val = np.reshape(X_val,(-1,22,tl,1))

    label0 = 769
    new_label0 = 0
    for i in range(4):
        m1 = (y_test==label0)
        m2 = (y_train==label0)
        m3 = (y_val==label0)
        np.place(y_test,m1,new_label0)
        np.place(y_train,m2,new_label0)
        np.place(y_val,m3,new_label0)
        label0 += 1
        new_label0 += 1

    labelNames = [0,1,2,3]
    train_set = {'data': X_train, 'labels': y_train}
    test_set = {'data': X_test, 'labels': y_test}
    val_set = {'data': X_val, 'labels': y_val}
    return train_set, test_set, val_set, labelNames

def sample_batch(dataset, batch_size):
    N = dataset['data'].shape[0]
    indices = np.random.randint(N, size=batch_size)
    return {key: dataset[key][indices] for key in dataset}

def train_model_gan2(target_dir,start,stop,step,time_bin,fsz,use_batchnorm,use_dropout,use_l2loss,num_class,max_step,log_frequency,batch_size,model_saving_freq):
    # load data
    print("Loading datasets...")
    trainSet, testSet, valSet, labelNames = data_loader_gan2()
    print("Dataset loading completes.")
    
    best_val_acc = 0
    best_val_path = target_dir
    


    # reset default graph
    tf.reset_default_graph()

    global_step = tf.train.get_or_create_global_step()
    
    model = Model(config, labelNames,start,stop,step,time_bin,fsz,use_batchnorm,use_dropout,use_l2loss)

    # training setups
    saver = tf.train.Saver(max_to_keep=100)

    # Build an initialization operation to run below.
    init = tf.global_variables_initializer()
    
    config_proto = tf.ConfigProto()

    off = rewriter_config_pb2.RewriterConfig.OFF
    config_proto.graph_options.rewrite_options.arithmetic_optimization = off
    
    sess = tf.Session(config=config_proto)

    sess.run(init)

    # main loop
    for step in range(max_step):
        start_time = time.time()

        # prepare data
        train_batch = sample_batch(trainSet, batch_size)
        # feed dict
        feed_dict = {
            model.input: train_batch['data'],
            model.fine_labels: train_batch['labels'],
            model.is_training: True,
            model.drprob: 0.7
        }
        
        fetch_list = [model.optimizer_op, model.loss, model.global_step]
        _, loss_value, global_step_value = sess.run(fetch_list, feed_dict=feed_dict)
        duration = time.time() - start_time
        assert not np.isnan(loss_value), 'Model diverged with loss = NaN'

        # log
        if step % log_frequency == 0:
            num_examples_per_step = batch_size
            examples_per_sec = num_examples_per_step / duration
            sec_per_batch = duration
            format_str = (
                '%s: step %d, examples %d, loss = %.9f (%.3f examples/sec; %.3f sec/batch)'
            )
            print(
                format_str % (
                    datetime.now(), step, batch_size * step,
                    loss_value,
                    examples_per_sec, sec_per_batch
                )
            )

        # Save the model checkpoint periodically.
        if step % model_saving_freq == 0 or (step + 1) == max_step:
            fetch_list = [model.accuracy,model.top_2_accuracy]
            feed_dict = {
                model.input: valSet['data'],
                model.fine_labels: valSet['labels'],
                model.is_training: False,
            }
            val_acc_1, val_acc_2 =  sess.run(fetch_list, feed_dict=feed_dict)
            checkpoint_path = os.path.join(target_dir, 'model.ckpt')
            saver.save(sess, checkpoint_path, global_step=int(global_step_value))
            print('Top 1 validation accuracy: {} and top 2 validation accuracy: {}'.format(val_acc_1,val_acc_2))
            print('Model Saved!')
            if val_acc_1>best_val_acc:
                best_val_acc = val_acc_1
                best_val_path = checkpoint_path+'-'+str(int(global_step_value))
    return best_val_acc,best_val_path


def test_model_gan2(model_dir,start,stop,step,time_bin,fsz,use_batchnorm,use_dropout,use_l2loss):
    print("Loading datasets...")
    trainSet, testSet, valSet, labelNames = data_loader_gan2()
    print("Dataset loading completes.")
    
    # reset default graph
    tf.reset_default_graph()
    
    # create model
    
    model = Model(config, labelNames,start,stop,step,time_bin,fsz,use_batchnorm,use_dropout,use_l2loss)
    # training setups
    saver = tf.train.Saver(max_to_keep=100)
    test_accuracy = [0, 0]
    
    config_proto = tf.ConfigProto()

    off = rewriter_config_pb2.RewriterConfig.OFF
    config_proto.graph_options.rewrite_options.arithmetic_optimization = off

    with tf.Session(config=config_proto) as session:
        session.run(tf.global_variables_initializer())
        saver.restore(session, model_dir)
        # test model
        exp_results = run_single_step(session, model, testSet, mode='test')
        test_accuracy[0] = exp_results['top_1_accuracy']
        test_accuracy[1] = exp_results['top_2_accuracy']
        print('top_1_accuracy_test = ', test_accuracy[0], 'top_2_accuracy_test = ', test_accuracy[1])
        
    top1_acc = test_accuracy[0]
    top2_acc = test_accuracy[1]
    return top1_acc,top2_acc


def run_single_step(
        session,
        model,
        batch,
        mode='test',
        log=True,
):
    # construct feed dict
    feed_dict = {
        model.input: batch['data'],
        # model.coarse_labels: batch['coarse_labels'],
        model.fine_labels: batch['labels'],
        # model.label_mapping: label_mapping,
        model.is_training: mode == 'train'
    }
    
    # select proper summary op
    if mode == 'train':
        summary_op = model.train_summary_op
    elif mode == 'val':
        summary_op = model.val_summary_op
    else:
        summary_op = model.test_summary_op
    
    # construct fetch list
    fetch_list = [model.global_step, summary_op, model.loss, model.accuracy, model.top_2_accuracy]

    # run single step
    _start_time = time.time()
    _step, _summary, _loss, _top_1, _top_2 = session.run(fetch_list, feed_dict=feed_dict)[:5]
    _end_time = time.time()
    
    # collect step statistics
    step_time = _end_time - _start_time
    batch_size = batch['data'].shape[0]
    
    # log in console
    if log:
        print(('[{:5s} step {:4d}] loss: {:.5f}; top_1_accuracy: {:.5f}; top_5_accuracy: {:5f} ' +
              '({:.3f} sec/batch; {:.3f} instances/sec)'
              ).format(mode, _step, _loss, _top_1, _top_2, 
                       step_time, batch_size / step_time))
    
    # log results to file and return statistics
    if mode == 'test':
        test_fetch_list = [model.per_class_accuracy,
                model.top_2_per_class_accuracy,
                model.confusion_matrix, 
                model.pred, model.probs]
        _top_1_c,  _top_2_c, _cm, _pred, _probs = \
                session.run(test_fetch_list, feed_dict=feed_dict)
        
        
        # Log detailed test results in pickle format
        stats = {
            "loss": _loss,
            "top_1_accuracy": _top_1,
            "top_2_accuracy": _top_2,
            "top_1_perclass_accuracy": _top_1_c,
            "top_2_perclass_accuracy": _top_2_c,
            "confusion_matrix": _cm,
            "pred": _pred,
            "probs": _probs
        }
    else:
        stats = {
            "step": _step,
            "loss": _loss,
            "top_1_accuracy": _top_1,
            "top_2_accuracy": _top_2
        }
        
    return stats


In [3]:
best_val_gan2 = {}
num_cls = 4
mstp = 5000
lfrq = 10
bsz = 50
msf = 100
tr = './trained_model_final/DCNN_GAN5'
if not os.path.exists(tr):
    os.mkdir(tr)
start = 0
stop = 250
step = 10
time_bin = 350
fsz = 3
use_batchnorm = True
use_dropout = True
use_l2loss = True
acc = 'Accuracy_gan4'
path = 'Path_gan4'

best_val_gan2[acc],best_val_gan2[path] = train_model_gan2(tr,start,stop,step,time_bin,fsz,use_batchnorm,use_dropout,use_l2loss,num_cls,mstp,lfrq,bsz,msf)

Loading datasets...
Dataset loading completes.
2019-03-19 00:23:15.886005: step 0, examples 0, loss = 1.420910001 (30.224 examples/sec; 1.654 sec/batch)
Top 1 validation accuracy: 0.24528302252292633 and top 2 validation accuracy: 0.5094339847564697
Model Saved!
2019-03-19 00:23:34.147170: step 10, examples 500, loss = 1.413841844 (35.108 examples/sec; 1.424 sec/batch)
2019-03-19 00:23:47.606177: step 20, examples 1000, loss = 1.416572094 (37.330 examples/sec; 1.339 sec/batch)
2019-03-19 00:24:01.245702: step 30, examples 1500, loss = 1.370634913 (36.621 examples/sec; 1.365 sec/batch)
2019-03-19 00:24:14.839348: step 40, examples 2000, loss = 1.469452024 (36.917 examples/sec; 1.354 sec/batch)
2019-03-19 00:24:28.398089: step 50, examples 2500, loss = 1.386072159 (36.621 examples/sec; 1.365 sec/batch)
2019-03-19 00:24:42.068530: step 60, examples 3000, loss = 1.402094603 (36.594 examples/sec; 1.366 sec/batch)
2019-03-19 00:24:55.486647: step 70, examples 3500, loss = 1.403366804 (36.972

2019-03-19 00:38:58.693691: step 670, examples 33500, loss = 1.221694946 (37.385 examples/sec; 1.337 sec/batch)
2019-03-19 00:39:12.264399: step 680, examples 34000, loss = 1.100214124 (37.695 examples/sec; 1.326 sec/batch)
2019-03-19 00:39:25.380323: step 690, examples 34500, loss = 1.153830647 (37.274 examples/sec; 1.341 sec/batch)
2019-03-19 00:39:38.561075: step 700, examples 35000, loss = 1.227276683 (38.505 examples/sec; 1.299 sec/batch)
Top 1 validation accuracy: 0.5400943160057068 and top 2 validation accuracy: 0.7547169923782349
Model Saved!
2019-03-19 00:39:56.279399: step 710, examples 35500, loss = 1.021826982 (37.865 examples/sec; 1.320 sec/batch)
2019-03-19 00:40:09.867063: step 720, examples 36000, loss = 1.297267437 (33.289 examples/sec; 1.502 sec/batch)
2019-03-19 00:40:24.176795: step 730, examples 36500, loss = 1.245475769 (35.682 examples/sec; 1.401 sec/batch)
2019-03-19 00:40:37.715588: step 740, examples 37000, loss = 1.194679737 (37.357 examples/sec; 1.338 sec/ba

2019-03-19 00:54:31.094936: step 1340, examples 67000, loss = 1.069013953 (37.469 examples/sec; 1.334 sec/batch)
2019-03-19 00:54:44.369435: step 1350, examples 67500, loss = 1.197281718 (37.951 examples/sec; 1.317 sec/batch)
2019-03-19 00:54:57.779574: step 1360, examples 68000, loss = 1.107256055 (37.553 examples/sec; 1.331 sec/batch)
2019-03-19 00:55:11.200682: step 1370, examples 68500, loss = 0.986317098 (37.638 examples/sec; 1.328 sec/batch)
2019-03-19 00:55:24.665674: step 1380, examples 69000, loss = 1.175650239 (36.999 examples/sec; 1.351 sec/batch)
2019-03-19 00:55:38.215438: step 1390, examples 69500, loss = 0.874410272 (37.865 examples/sec; 1.320 sec/batch)
2019-03-19 00:55:51.632557: step 1400, examples 70000, loss = 1.228473067 (36.972 examples/sec; 1.352 sec/batch)
Top 1 validation accuracy: 0.5754716992378235 and top 2 validation accuracy: 0.775943398475647
Model Saved!
2019-03-19 00:56:09.949573: step 1410, examples 70500, loss = 1.194548011 (36.917 examples/sec; 1.354

Top 1 validation accuracy: 0.6084905862808228 and top 2 validation accuracy: 0.7995283007621765
Model Saved!
2019-03-19 01:10:50.811579: step 2010, examples 100500, loss = 1.095290899 (34.432 examples/sec; 1.452 sec/batch)
2019-03-19 01:11:05.417368: step 2020, examples 101000, loss = 1.180123448 (33.307 examples/sec; 1.501 sec/batch)
2019-03-19 01:11:20.138054: step 2030, examples 101500, loss = 0.960664749 (33.531 examples/sec; 1.491 sec/batch)
2019-03-19 01:11:34.808930: step 2040, examples 102000, loss = 1.021329641 (33.736 examples/sec; 1.482 sec/batch)
2019-03-19 01:11:49.103398: step 2050, examples 102500, loss = 1.090866566 (35.194 examples/sec; 1.421 sec/batch)
2019-03-19 01:12:03.372631: step 2060, examples 103000, loss = 1.017120838 (32.622 examples/sec; 1.533 sec/batch)
2019-03-19 01:12:18.583178: step 2070, examples 103500, loss = 1.002319455 (33.828 examples/sec; 1.478 sec/batch)
2019-03-19 01:12:35.327384: step 2080, examples 104000, loss = 1.031033993 (31.421 examples/s

2019-03-19 01:26:59.895481: step 2670, examples 133500, loss = 0.986196876 (38.949 examples/sec; 1.284 sec/batch)
2019-03-19 01:27:12.714234: step 2680, examples 134000, loss = 0.879651070 (38.767 examples/sec; 1.290 sec/batch)
2019-03-19 01:27:25.758889: step 2690, examples 134500, loss = 1.057764888 (37.598 examples/sec; 1.330 sec/batch)
2019-03-19 01:27:38.648621: step 2700, examples 135000, loss = 0.821721673 (39.395 examples/sec; 1.269 sec/batch)
Top 1 validation accuracy: 0.625 and top 2 validation accuracy: 0.8349056839942932
Model Saved!
2019-03-19 01:27:57.424478: step 2710, examples 135500, loss = 0.951138556 (37.558 examples/sec; 1.331 sec/batch)
2019-03-19 01:28:10.305757: step 2720, examples 136000, loss = 0.946308196 (40.005 examples/sec; 1.250 sec/batch)
2019-03-19 01:28:23.310826: step 2730, examples 136500, loss = 0.942223310 (39.259 examples/sec; 1.274 sec/batch)
2019-03-19 01:28:36.244983: step 2740, examples 137000, loss = 0.938438833 (38.426 examples/sec; 1.301 sec

2019-03-19 01:41:54.834005: step 3330, examples 166500, loss = 0.619890273 (38.431 examples/sec; 1.301 sec/batch)
2019-03-19 01:42:07.880419: step 3340, examples 167000, loss = 0.649988234 (39.095 examples/sec; 1.279 sec/batch)
2019-03-19 01:42:22.662330: step 3350, examples 167500, loss = 0.682064295 (33.223 examples/sec; 1.505 sec/batch)
2019-03-19 01:42:36.586238: step 3360, examples 168000, loss = 0.829471231 (38.956 examples/sec; 1.284 sec/batch)
2019-03-19 01:42:50.591303: step 3370, examples 168500, loss = 0.758712590 (35.811 examples/sec; 1.396 sec/batch)
2019-03-19 01:43:06.282114: step 3380, examples 169000, loss = 0.642302752 (29.301 examples/sec; 1.706 sec/batch)
2019-03-19 01:43:23.151577: step 3390, examples 169500, loss = 0.812564909 (32.122 examples/sec; 1.557 sec/batch)
2019-03-19 01:43:37.640701: step 3400, examples 170000, loss = 0.691053629 (35.429 examples/sec; 1.411 sec/batch)
Top 1 validation accuracy: 0.6132075190544128 and top 2 validation accuracy: 0.830188691

2019-03-19 01:58:04.508968: step 4000, examples 200000, loss = 0.581089735 (37.575 examples/sec; 1.331 sec/batch)
Top 1 validation accuracy: 0.6391509175300598 and top 2 validation accuracy: 0.8349056839942932
Model Saved!
2019-03-19 01:58:22.291182: step 4010, examples 200500, loss = 0.717105210 (38.565 examples/sec; 1.297 sec/batch)
2019-03-19 01:58:35.407517: step 4020, examples 201000, loss = 0.582926333 (38.267 examples/sec; 1.307 sec/batch)
2019-03-19 01:58:48.486406: step 4030, examples 201500, loss = 0.758684099 (38.671 examples/sec; 1.293 sec/batch)
2019-03-19 01:59:01.669483: step 4040, examples 202000, loss = 0.629420817 (36.478 examples/sec; 1.371 sec/batch)
2019-03-19 01:59:14.818870: step 4050, examples 202500, loss = 0.751155436 (39.743 examples/sec; 1.258 sec/batch)
2019-03-19 01:59:27.819203: step 4060, examples 203000, loss = 0.546702325 (38.742 examples/sec; 1.291 sec/batch)
2019-03-19 01:59:41.036253: step 4070, examples 203500, loss = 0.623867452 (38.229 examples/s

2019-03-19 02:13:52.398462: step 4660, examples 233000, loss = 0.572956502 (34.583 examples/sec; 1.446 sec/batch)
2019-03-19 02:14:07.042555: step 4670, examples 233500, loss = 0.534530997 (34.634 examples/sec; 1.444 sec/batch)
2019-03-19 02:14:21.695974: step 4680, examples 234000, loss = 0.732063234 (34.618 examples/sec; 1.444 sec/batch)
2019-03-19 02:14:36.789676: step 4690, examples 234500, loss = 0.600407362 (28.789 examples/sec; 1.737 sec/batch)
2019-03-19 02:14:54.040891: step 4700, examples 235000, loss = 0.473254591 (28.991 examples/sec; 1.725 sec/batch)
Top 1 validation accuracy: 0.6297169923782349 and top 2 validation accuracy: 0.8349056839942932
Model Saved!
2019-03-19 02:15:18.200281: step 4710, examples 235500, loss = 0.655471861 (28.812 examples/sec; 1.735 sec/batch)
2019-03-19 02:15:35.087882: step 4720, examples 236000, loss = 0.544600487 (26.207 examples/sec; 1.908 sec/batch)
2019-03-19 02:15:51.928088: step 4730, examples 236500, loss = 0.451531321 (30.283 examples/s

In [5]:
# Test model accuracy
start = 0
stop = 250
step = 10
time_bin = 350
#Nb = BATCH_SIZE
fsz = 3
use_batchnorm = True
use_dropout = True
use_l2loss = True
path = 'Path_gan4' 
model_dir = best_val_gan2[path]
print('Test Accuracy for all subjects:')
top_1_acc_test, top_2_acc_test = test_model_gan2(model_dir,start,stop,step,time_bin,fsz,use_batchnorm,use_dropout,use_l2loss)

Test Accuracy for all subjects:
Loading datasets...
Dataset loading completes.
INFO:tensorflow:Restoring parameters from ./trained_model_final/DCNN_GAN5\model.ckpt-4801
[test  step 4801] loss: 1.23245; top_1_accuracy: 0.63431; top_5_accuracy: 0.828442 (7.926 sec/batch; 111.791 instances/sec)
top_1_accuracy_test =  0.6343115 top_2_accuracy_test =  0.82844245
